In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import yfinance as yf
from datetime import datetime
from dateutil.relativedelta import relativedelta

###Question 1: [Index] S&P 500 Stocks Added to the Index

####Which year had the highest number of additions?

In [2]:
url = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"
tables = pd.read_html(url)
df = tables[0]
print(df.info())
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 8 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   Symbol                 503 non-null    object
 1   Security               503 non-null    object
 2   GICS Sector            503 non-null    object
 3   GICS Sub-Industry      503 non-null    object
 4   Headquarters Location  503 non-null    object
 5   Date added             503 non-null    object
 6   CIK                    503 non-null    int64 
 7   Founded                503 non-null    object
dtypes: int64(1), object(7)
memory usage: 31.6+ KB
None


,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date added,CIK,Founded
0,MMM,3M,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1957-03-04,66740,1902
1,AOS,A. O. Smith,Industrials,Building Products,"Milwaukee, Wisconsin",2017-07-26,91142,1916
2,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1957-03-04,1800,1888
3,ABBV,AbbVie,Health Care,Biotechnology,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
4,ACN,Accenture,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989


In [3]:
# Data Added is the column that is useful for us
df = df[['Symbol', 'Date added']]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 503 entries, 0 to 502
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Symbol      503 non-null    object
 1   Date added  503 non-null    object
dtypes: object(2)
memory usage: 8.0+ KB


In [4]:
# Extracting Year from Data Added
df["Year Added"] = pd.to_datetime(df["Date added"], errors='coerce').dt.year

<ipython-input-4-02e083d74a71>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["Year Added"] = pd.to_datetime(df["Date added"], errors='coerce').dt.year


In [5]:
df.head()

,Symbol,Date added,Year Added
0,MMM,1957-03-04,1957
1,AOS,2017-07-26,2017
2,ABT,1957-03-04,1957
3,ABBV,2012-12-31,2012
4,ACN,2011-07-06,2011


In [6]:
years_df = df["Year Added"].dropna().astype(int)
years_filtered = years_df[years_df != 1957]        # S&P 500 Index started in 1957
additions_per_year = years_filtered.value_counts()
additions_per_year.head()

,count
Year Added,
2017,23
2016,23
2019,22
2008,17
2024,16


Answer: 2017 has the highest number of additions to S&P 500 with 23 additions

####How many current S&P 500 stocks have been in the index for more than 20 years?

In [7]:
components_20y_older = df[df['Year Added'] <= 2005]
print(f"Number of Companies which have been part of S&P 500 for more than 20 years = {components_20y_older.shape[0]}")

Number of Companies which have been part of S&P 500 for more than 20 years = 226


Additionally: There are 226 companies which are part of the index for more than 20 years.

###Question 2: [Macro] Indexes YTD (as of 1 May 2025)

####How many indexes (out of 10) have better year-to-date returns than the US (S&P 500) as of May 1, 2025?

In [8]:
def calculate_return(symbol, start_date, end_date):
    """
    Downloads the Close prices and returns a Python float for:
      (last_close / first_close - 1) * 100
    or None if data is missing.
    """
    df = yf.download(symbol, start=start_date, end=end_date, progress=False)
    if df.empty:
        return None
    # grab the first and last closing price
    first = df['Close'].iloc[0]
    last  = df['Close'].iloc[-1]
    # compute and force to float
    return float((last.iloc[0] / first.iloc[0] - 1) * 100)

In [9]:
# Defining Indices that will be compared to S&P 500
indices = [
    ('^GSPC', 'United States'),
    ('000001.SS', 'China'),
    ('^HSI', 'Hong Kong'),
    ('^AXJO', 'Australia'),
    ('^NSEI', 'India'),
    ('^GSPTSE', 'Canada'),
    ('^GDAXI', 'Germany'),
    ('^FTSE', 'United Kingdom'),
    ('^N225', 'Japan'),
    ('^MXX', 'Mexico'),
    ('^BVSP', 'Brazil')
]

horizons = {
    'YTD':   ('2025-01-01', '2025-05-02'),
    '1Y':    None,   # will fill in loop
    '3Y':    None,
    '5Y':    None,
    '10Y':   None
}

end_date = '2025-05-02'
as_of = datetime(2025, 5, 1)

In [10]:
for label in ['1Y','3Y','5Y','10Y']:
    years = int(label.replace('Y',''))
    start = (as_of - relativedelta(years=years)).strftime('%Y-%m-%d')
    horizons[label] = (start, end_date)

In [11]:
horizons

{'YTD': ('2025-01-01', '2025-05-02'),
 '1Y': ('2024-05-01', '2025-05-02'),
 '3Y': ('2022-05-01', '2025-05-02'),
 '5Y': ('2020-05-01', '2025-05-02'),
 '10Y': ('2015-05-01', '2025-05-02')}

In [12]:
all_returns = {label: {} for label in horizons}

for label, (start, end) in horizons.items():
    for ticker, country in indices:
        ret = calculate_return(ticker, start, end)
        all_returns[label][country] = ret

YF.download() has changed argument auto_adjust default to True


In [13]:
for label in horizons:
    us_retr = all_returns[label]['United States']
    print(f"\n[{label}] S&P 500 return = {us_retr:.2f}%")

    # List of Indices which have better return
    beats = []
    for country, retr in all_returns[label].items():
        if country == 'United States' or retr is None or us_retr is None:
            continue
        # Comparing to check if Index returns are better
        if retr > us_retr:
            beats.append((country, retr))

    print(f"Number beating S&P 500: {len(beats)}")
    for country, retr in sorted(beats, key=lambda x: x[1], reverse=True):
        print(f"  {country:<14} {retr:.2f}%")


[YTD] S&P 500 return = -4.51%
Number beating S&P 500: 9
  Mexico         13.05%
  Hong Kong      12.72%
  Brazil         12.44%
  Germany        12.35%
  United Kingdom 2.87%
  India          2.49%
  China          0.50%
  Canada         -0.41%
  Australia      -0.68%

[1Y] S&P 500 return = 11.67%
Number beating S&P 500: 3
  Germany        25.71%
  Hong Kong      21.49%
  Canada         14.12%

[3Y] S&P 500 return = 34.86%
Number beating S&P 500: 3
  Germany        61.40%
  India          42.56%
  Japan          35.92%

[5Y] S&P 500 return = 97.98%
Number beating S&P 500: 2
  India          161.84%
  Germany        114.94%

[10Y] S&P 500 return = 165.81%
Number beating S&P 500: 1
  India          192.06%


Answer: There are 9 Indices which have better YTD returns than S&P 500 as of 1st May 2025

####How many of these indexes have better returns than the S&P 500 over 3, 5, and 10 year periods?

[1Y] S&P 500 return = 11.67\
Number beating S&P 500: **3**\
  Germany       25.71%\
  Hong Kong     21.49%\
  Canada        14.12%

\
[3Y] S&P 500 return = 34.86\
Number beating S&P 500: **3**\
  Germany       61.40%\
  India         42.56%\
  Japan         35.92%

\
[5Y] S&P 500 return = 97.97\
Number beating S&P 500: **2**\
  India        161.84%\
  Germany      114.94%

\
[10Y] S&P 500 return = 165.8\
Number beating S&P 500: **1**\
  India        192.06%

###Question 3: [Index] S&P 500 Market Corrections Analysis

####Calculate the median duration (in days) of significant market corrections in the S&P 500 index.

In [14]:
sp500 = yf.download("^GSPC", start="1950-01-01", progress=False)[['Close']].dropna()
sp500.columns = ['_'.join(col).strip() for col in sp500.columns.values]

sp500_df = sp500.reset_index()
sp500_df['Date'] = pd.to_datetime(sp500_df['Date'])
sp500_df = sp500_df.rename(columns={'Date': 'Date', 'Close_^GSPC': 'Close'})

sp500_df

,Date,Close
0,1950-01-03,16.660000
1,1950-01-04,16.850000
2,1950-01-05,16.930000
3,1950-01-06,16.980000
4,1950-01-09,17.080000
...,...,...
18970,2025-05-28,5888.549805
18971,2025-05-29,5912.169922
18972,2025-05-30,5911.689941
18973,2025-06-02,5935.939941


In [15]:
sp500_df['all_time_high'] = sp500_df['Close'].cummax()
sp500_df['new_high'] = sp500_df['Close'] == sp500_df['all_time_high']
highs = sp500_df[sp500_df['new_high']]
highs

,Date,Close,all_time_high,new_high
0,1950-01-03,16.660000,16.660000,True
1,1950-01-04,16.850000,16.850000,True
2,1950-01-05,16.930000,16.930000,True
3,1950-01-06,16.980000,16.980000,True
4,1950-01-09,17.080000,17.080000,True
...,...,...,...,...
18852,2024-12-04,6086.490234,6086.490234,True
18854,2024-12-06,6090.270020,6090.270020,True
18884,2025-01-23,6118.709961,6118.709961,True
18901,2025-02-18,6129.580078,6129.580078,True


In [16]:
corrections = []

for i in range(1, len(highs)):
    start = highs.iloc[i - 1]['Date']
    end = highs.iloc[i]['Date']
    between = sp500_df[(sp500_df['Date'] >= start) & (sp500_df['Date'] <= end)]

    if between.empty or len(between) < 2: # skip if there a no values in between or steps are less than 2
        continue

    min_price = between['Close'].min()
    min_row = between[between['Close'] == min_price].iloc[0]
    min_date = min_row['Date']

    peak_price = between['Close'].iloc[0]
    drawdown = (peak_price - min_price) / peak_price * 100
    duration = (min_date - start).days

    if drawdown >= 5: # we need corrections more than 5%
        corrections.append({
            'Start': start,
            'Low': min_date,
            'End': end,
            'Drawdown (%)': round(drawdown, 2),
            'Duration (days)': duration
        })

df_corrections = pd.DataFrame(corrections)

df_corrections

,Start,Low,End,Drawdown (%),Duration (days)
0,1950-06-12,1950-07-17,1950-09-22,14.02,35
1,1950-11-24,1950-12-04,1950-12-28,6.50,10
2,1951-05-03,1951-06-29,1951-08-02,8.11,57
3,1951-10-15,1951-11-23,1952-01-03,6.08,39
4,1952-01-22,1952-02-20,1952-06-25,6.37,29
...,...,...,...,...,...
66,2020-09-02,2020-09-23,2020-11-13,9.60,21
67,2021-09-02,2021-10-04,2021-10-21,5.21,32
68,2022-01-03,2022-10-12,2024-01-19,25.43,282
69,2024-03-28,2024-04-19,2024-05-15,5.46,22


In [17]:
percentiles = df_corrections['Duration (days)'].quantile([0, 0.25, 0.5, 0.75, 1])
percentiles

,Duration (days)
0.00,7.0
0.25,21.5
0.50,39.0
0.75,89.0
1.00,929.0


Answer: Median duration of significant market corrections = 39

###Question 4: [Stocks] Earnings Surprise Analysis for Amazon (AMZN)

####Calculate the median 2-day percentage change in stock prices following positive earnings surprises days.

In [18]:
amzn_earnings_df  = pd.read_csv("ha1_Amazon.csv", delimiter=';')

In [19]:
amzn_earnings_df['Earnings Date'] = amzn_earnings_df['Earnings Date'].astype(str).str.split(' at ', expand=True)[0]
amzn_earnings_df['Earnings Date'] = pd.to_datetime(amzn_earnings_df['Earnings Date'], errors='coerce')
amzn_earnings_df

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
0,AMZN,Amazon.com Inc,2026-04-29,-,-,-
1,AMZN,Amazon.com Inc,2026-02-04,-,-,-
2,AMZN,Amazon.com Inc,2025-10-29,-,-,-
3,AMZN,Amazon.com Inc,2025-07-30,-,-,-
4,AMZN,"Amazon.com, Inc.",2025-05-01,???.36,???.59,+16.74
...,...,...,...,...,...,...
112,AMZN,"Amazon.com, Inc.",1998-04-27,-,-,+13.92
113,AMZN,"Amazon.com, Inc.",1998-01-22,-,-,+11.41
114,AMZN,"Amazon.com, Inc.",1997-10-27,-,-,+13.29
115,AMZN,"Amazon.com, Inc.",1997-07-10,-,-,+13.33


In [20]:
numeric_cols = ["EPS Estimate", "Reported EPS"]
for col in numeric_cols:
  amzn_earnings_df[col] = pd.to_numeric(amzn_earnings_df[col], errors="coerce")
  amzn_earnings_df.dropna(subset=numeric_cols + ['Earnings Date'], inplace=True)
amzn_earnings_df

,Symbol,Company,Earnings Date,EPS Estimate,Reported EPS,Surprise (%)
8,AMZN,"Amazon.com, Inc.",2024-04-30,0.83,0.98,+17.91
9,AMZN,"Amazon.com, Inc.",2024-02-01,0.80,1.00,+24.55
10,AMZN,"Amazon.com, Inc.",2023-10-26,0.58,0.94,+60.85
11,AMZN,"Amazon.com, Inc.",2023-08-03,0.35,0.65,+85.73
12,AMZN,"Amazon.com, Inc.",2023-04-27,0.21,0.31,+46.36
...,...,...,...,...,...,...
104,AMZN,"Amazon.com, Inc.",2000-04-26,-0.02,-0.02,+3.79
105,AMZN,"Amazon.com, Inc.",2000-02-02,-0.02,-0.03,-13.36
106,AMZN,"Amazon.com, Inc.",1999-10-27,-0.01,-0.01,+7.54
107,AMZN,"Amazon.com, Inc.",1999-07-21,-0.01,-0.01,+0.47


In [21]:
# Filter for positive earnings surprises and set 'Earnings Date' as index
amzn_earnings_pos_surp_df = amzn_earnings_df.loc[amzn_earnings_df["Reported EPS"] > amzn_earnings_df["EPS Estimate"]].copy()
amzn_earnings_pos_surp_df.set_index('Earnings Date', inplace=True) # Set Earnings Date as index for easy lookup
amzn_earnings_pos_surp_df.sort_index(inplace=True) # Good practice to sort by date
print(f"\nNumber of positive earnings surprises identified: {len(amzn_earnings_pos_surp_df)}")


Number of positive earnings surprises identified: 33


In [22]:
amzn_ticker = 'AMZN'
ticker_obj = yf.Ticker(amzn_ticker)
amzn_price_data = ticker_obj.history(period="max", interval="1d")
amzn_price_data

,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,
1997-05-15 00:00:00-04:00,0.121875,0.125000,0.096354,0.097917,1443120000,0.0,0.0
1997-05-16 00:00:00-04:00,0.098438,0.098958,0.085417,0.086458,294000000,0.0,0.0
1997-05-19 00:00:00-04:00,0.088021,0.088542,0.081250,0.085417,122136000,0.0,0.0
1997-05-20 00:00:00-04:00,0.086458,0.087500,0.081771,0.081771,109344000,0.0,0.0
1997-05-21 00:00:00-04:00,0.081771,0.082292,0.068750,0.071354,377064000,0.0,0.0
...,...,...,...,...,...,...,...
2025-05-28 00:00:00-04:00,205.919998,207.660004,204.410004,204.720001,28549800,0.0,0.0
2025-05-29 00:00:00-04:00,208.029999,208.809998,204.229996,205.699997,34650000,0.0,0.0
2025-05-30 00:00:00-04:00,204.839996,205.990005,201.699997,205.009995,51679400,0.0,0.0


In [23]:
amzn_price_data.index = amzn_price_data.index.date

if isinstance(amzn_price_data.columns, pd.MultiIndex):
    if 'Close' in amzn_price_data.columns:
        amzn_price_data['Close_Single'] = amzn_price_data['Close']
    else:
        print("Warning: 'Close' column not found directly in yfinance data.")
        amzn_price_data['Close_Single'] = amzn_price_data[('Close', amzn_ticker)] if (('Close', amzn_ticker) in amzn_price_data.columns) else amzn_price_data['Close']
else:
    amzn_price_data['Close_Single'] = amzn_price_data['Close']

In [24]:
amzn_price_data["2days_return"] = (amzn_price_data["Close_Single"].shift(-1) / amzn_price_data["Close_Single"].shift(1) - 1)
amzn_price_data = amzn_price_data[["Close_Single", "2days_return"]].copy()
amzn_price_data

,Close_Single,2days_return
1997-05-15,0.097917,NaN
1997-05-16,0.086458,-0.127659
1997-05-19,0.085417,-0.054211
1997-05-20,0.081771,-0.164639
1997-05-21,0.071354,-0.146494
...,...,...
2025-05-28,204.720001,-0.001553
2025-05-29,205.699997,0.001417
2025-05-30,205.009995,0.004618
2025-06-02,206.649994,0.004829


In [25]:
returns_after_surprise = amzn_price_data.loc[amzn_earnings_pos_surp_df.index, "2days_return"].dropna()

median_surprise = returns_after_surprise.median()
all_returns = amzn_price_data["2days_return"].dropna()
median_all = all_returns.median()

print("\n--- Results ---")
print("Median 2-day return after positive surprises:", round(median_surprise * 100, 1), "%")
print("Median 2-day return overall:", round(median_all * 100, 1), "%")


--- Results ---
Median 2-day return after positive surprises: 2.6 %
Median 2-day return overall: 0.2 %


Answer: The median 2-day percentage change in stock prices following positive earnings surprises days for Amazon = 2.6%